In [73]:
import copy

class Graph:
    def __init__(self, vertices, source):
        self.source = source
        self.vertices = vertices
        self.INFINITY = 999999999
        self.edges_vertices = {vertex: [0, {}] for vertex in range(vertices + 1)}
        self.directions = {vertex: set() for vertex in range(vertices + 1)}
        self.visited = set()
        self.topological_sort = []

    def set_init_distances(self):
        for vertex in range(len(self.edges_vertices)):
            if vertex == 0:
                continue
            else:
                if vertex == self.source:
                    self.edges_vertices[vertex][0] = 0
                else:
                    self.edges_vertices[vertex][0] = self.INFINITY

    def add_edge(self, nodea, nodeb, weight):
        self.edges_vertices[nodea][1].update({ nodeb: (weight, self.INFINITY)})
        self.directions[nodea].add(nodeb)

    def get_edge(self, nodea, nodeb):
        return self.edges_vertices[nodea][1][nodeb]
    
    def get_graph(self):
        return self.edges_vertices

    def get_topological_sort(self):
        return self.topological_sort

    def get_visited(self):
        return self.visited

    def validation(self):
        for key in self.directions.keys():
            if len(self.directions[key]) > 0:
                if key not in self.visited:
                    self.visited.add(key)
                    self.dfs_traversal(key)
    
    def dfs_traversal(self, node):
        for neighbor in self.directions[node]:
            if neighbor not in self.visited:
                self.visited.add(neighbor)
                self.dfs_traversal(neighbor)
        self.topological_sort.append(node)

    def curate_graph(self, unreachable_numbers):
        new_graph = copy.deepcopy(g.get_graph())
        for cant_reach in no_puedo_alcanzar:
            del new_graph[cant_reach]
        return new_graph

    def bellman_ford(self, curated_graph, unreachable_list):
        new_vertices = max(curated_graph)
        for _ in range(len(curated_graph)):
            if _ in unreachable_list:
                continue
            else:
                for vertex in range(1, new_vertices):
                    if vertex not in unreachable_list:
                        adjs = curated_graph[vertex][1]
                        if len(adjs) > 0:
                            for destination in adjs:
                                self.relaxation(vertex, destination, curated_graph)

    def relaxation(self, nodea, nodeb, curated_graph):
        nodea_w = curated_graph[nodea][0]
        edge = curated_graph[nodea][1][nodeb]
        
        edge_w = edge[0]
        nodeb_w = edge[1]

        if nodea_w != self.INFINITY:
            if nodea_w + edge_w < nodeb_w:
                relaxed = (edge_w, nodea_w + edge_w)
                curated_graph[nodea][1][nodeb] = relaxed
                curated_graph[nodeb][0] = nodea_w + edge_w
                self.update_weight(nodeb, nodea_w + edge_w, curated_graph)
                return True
        return False

    def update_weight(self, node, weight, curated_graph):
        for vertex in curated_graph:
            neighbors = curated_graph[vertex][1]
            if node in neighbors:
                neighbors[node] = (neighbors[node][0], weight)

    def is_reachable(self, graph, start, visited = None):
        if visited is None:
            visited = set()
        visited.add(start)
        adjs = graph[start]

        for key in set(adjs) - visited:
            self.is_reachable(graph, key, visited)
        return visited

    def get_graph(self):
        return self.edges_vertices

    def get_simple_graph(self, curated_graph, unreachable_list):
        new_vertices = max(curated_graph)
        graph = {vertex: set([]) for vertex in range(new_vertices + 1)}

        for vertex in range(len(curated_graph)):
            if vertex not in unreachable_list:
                for adj in curated_graph[vertex][1].keys():
                    graph[vertex].add(adj)
        #print(f'curated graph {curated_graph}')
        print(f'this is the simple graph {graph}')
        return graph


    def get_distances(self, curated_graph, unreachable_list):
        s = ""
        print(f'curated graph {curated_graph}')
        destinations = self.is_reachable(self.get_simple_graph(curated_graph, unreachable_list), self.source)
        print(str(destinations))
        for vertex in range(1, len(curated_graph) + 1):
            if vertex in destinations:
                s += f"{curated_graph[vertex][0]} "
            else:
                s += "x "
        return s

In [74]:
with open("test_sdag.txt", "r") as f:
    vertices, edges = map(int, f.readline().split())
    source = 1
    g = Graph(vertices, source)

    while True:
        line = f.readline()
        if not line:
            break
        nodea, nodeb, weight = map(int, line.split())
        g.add_edge(nodea, nodeb, weight)
    
    g.set_init_distances()
    g.validation()
    top_list = g.get_topological_sort()[::-1]
    stopping_index = top_list.index(1)
    unreachable_list = top_list[:stopping_index]
    curated_graph = g.curate_graph(no_puedo_alcanzar)
    g.bellman_ford(curated_graph, unreachable_list)
    result = g.get_distances(curated_graph, unreachable_list).rstrip()

"""with open("result.txt", "w") as f:
    f.write(result)"""

curated graph {0: [0, {}], 1: [0, {4: (1, 1), 5: (-3, -3)}], 3: [-1, {}], 4: [1, {3: (-2, -1)}], 5: [-3, {4: (1, 1)}]}
this is the simple graph {0: set(), 1: {4, 5}, 2: set(), 3: set(), 4: {3}, 5: set()}
{1, 3, 4, 5}


'with open("result.txt", "w") as f:\n    f.write(result)'

In [64]:
result

'0 x -1 1 -3'

In [ ]:
{
    0: [0, {}], 
    1: [0, {4: (1, 1), 5: (-3, -3)}], 
    3: [-1, {}], 
    4: [1, {3: (-2, -1)}], 
    5: [-3, {4: (1, 1)}]
}

curated graph {
    0: [0, {}], 
    1: [0, {4: (1, 1), 5: (-3, -3)}], 
    3: [-1, {}], 4: [1, {3: (-2, -1)}], 
    5: [-3, {4: (1, 1)}]
    }